In [42]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [43]:
df = pd.read_csv(r'C:\Users\mahat\OneDrive\Desktop\Optimizing-Household-Electricity-through-Machine-Learning-main\Datasets\final_electricity_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
# df = df[df['Date'].dt.year == 2021]
df.reset_index(drop=True, inplace=True)

In [44]:
df

,Date,Total_Consumption
0,2020-01-01,10.6
1,2020-01-02,11.5
2,2020-01-03,24.3
3,2020-01-04,12.1
4,2020-01-05,8.5
...,...,...
1435,2023-12-27,12.4
1436,2023-12-28,8.5
1437,2023-12-29,2.5
1438,2023-12-30,5.1


In [45]:
df.tail()

,Date,Total_Consumption
1435,2023-12-27,12.4
1436,2023-12-28,8.5
1437,2023-12-29,2.5
1438,2023-12-30,5.1
1439,2023-12-31,11.4


In [46]:
df.shape

(1440, 2)

In [47]:
import math
import statsmodels.api as sm
import statsmodels.tsa.api as smt
# from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import matplotlib.pyplot as plt

actual_vals = df.Total_Consumption.values
actual_log = np.log10(actual_vals)

train, test = actual_vals[0:-80], actual_vals[-80:]
train_log, test_log = np.log10(train), np.log10(test)
my_order = (1, 1, 1)
my_seasonal_order = (1, 1, 1, 7)

In [48]:
history = [x for x in train_log]
predictions = list()
predict_log=list()
for t in range(len(test_log)):
    model = sm.tsa.SARIMAX(history, order=my_order, seasonal_order=my_seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    predict_log.append(output[0])
    yhat = 10**output[0]
    predictions.append(yhat)
    # obs = test_log[t]
    # history.append(obs)
   # print('predicted=%f, expected=%f' % (output[0], obs))
#error = math.sqrt(mean_squared_error(test_log, predict_log))
#print('Test rmse: %.3f' % error)
# plot
# figsize=(12, 7)
# plt.figure(figsize=figsize)
# pyplot.plot(test,label='Actuals')
# pyplot.plot(predictions, color='red',label='Predicted')
# pyplot.legend(loc='upper right')
# pyplot.show()

In [49]:
import pickle 
with open('module.pkl','wb') as f:
    pickle.dump(model_fit,f)

In [50]:
df_preds = df[-80:]
df_preds['Predicted_Consumption'] = predictions
df_preds

,Date,Total_Consumption,Predicted_Consumption
1360,2023-10-13,23.4,27.115414
1361,2023-10-14,25.8,27.115414
1362,2023-10-15,54.9,27.115414
1363,2023-10-16,24.6,27.115414
1364,2023-10-17,39.9,27.115414
...,...,...,...
1435,2023-12-27,12.4,27.115414
1436,2023-12-28,8.5,27.115414
1437,2023-12-29,2.5,27.115414
1438,2023-12-30,5.1,27.115414


In [52]:
import plotly.graph_objects as go
import plotly.offline as py

In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_preds['Date'], y=df_preds['Total_Consumption'],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=df_preds['Date'], y=df_preds['Predicted_Consumption'],
                    mode='lines',
                    name='lines'))

In [54]:
new_dates = pd.date_range(start='2024/01/01', end='2024/09/30')

In [55]:
train = df.Total_Consumption.values

my_order = (1, 1, 1)
my_seasonal_order = (1, 1, 1, 62)

model = sm.tsa.SARIMAX(train, order=my_order, seasonal_order=my_seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
model_fit = model.fit()
output = model_fit.forecast(62)

In [56]:
listofzeros = [np.nan] * 62

In [58]:
# Ensure new_dates is a list of valid date values
new_dates = pd.date_range(start="2024-01-01", periods=len(output), freq='D').tolist()

# Ensure listofzeros has the same length as output
listofzeros = [0] * len(output)

# Ensure output is a list or array of numerical values
if isinstance(output, pd.Series):
    output = output.tolist()

# Create the DataFrame with properly formatted data
data_new = {
    'Date': new_dates,
    'Total_Consumption': listofzeros,
    'Predicted_Consumption': output
}

df_new_preds = pd.DataFrame(data_new)


In [59]:
df_all = pd.concat([df_preds, df_new_preds], ignore_index=True)

In [60]:
df_all

,Date,Total_Consumption,Predicted_Consumption
0,2023-10-13,23.4,27.115414
1,2023-10-14,25.8,27.115414
2,2023-10-15,54.9,27.115414
3,2023-10-16,24.6,27.115414
4,2023-10-17,39.9,27.115414
...,...,...,...
137,2024-02-27,0.0,6.039189
138,2024-02-28,0.0,7.245652
139,2024-02-29,0.0,4.782189
140,2024-03-01,0.0,7.419251


In [61]:
df_all['MAE'] = df_all['Total_Consumption'] - df_all['Predicted_Consumption']

### Threshold: 5

In [62]:
df_anoms = df_all[df_all['MAE'] >= 5]
df_anoms.reset_index(drop=True, inplace=True)

In [63]:
df_anoms.tail(2)

,Date,Total_Consumption,Predicted_Consumption,MAE
16,2023-11-29,53.4,27.115414,26.284586
17,2023-11-30,44.1,27.115414,16.984586


In [64]:
df_anoms['extra_MAE'] = df_anoms['MAE'] - 5

In [65]:
df_anoms['extra_MAE_cost'] = df_anoms['extra_MAE']*9

In [66]:
df_anoms.groupby(df_anoms['Date'].dt.month)['extra_MAE_cost'].sum()

Date
10     765.967642
11    1731.735283
Name: extra_MAE_cost, dtype: float64

In [67]:
df_all.groupby(df_all['Date'].dt.month)['Total_Consumption'].sum().reset_index()['Total_Consumption']*7

0       0.0
1       0.0
2       0.0
3    3960.6
4    6951.0
5    2346.4
Name: Total_Consumption, dtype: float64

In [68]:
180/2300 # 8-15%

0.0782608695652174

### Threshold: 10

In [69]:
df_anoms = df_all[df_all['MAE'] >= 10]
df_anoms.reset_index(drop=True, inplace=True)

In [70]:
df_anoms.tail(2)

,Date,Total_Consumption,Predicted_Consumption,MAE
14,2023-11-29,53.4,27.115414,26.284586
15,2023-11-30,44.1,27.115414,16.984586


In [71]:
df_anoms['extra_MAE'] = df_anoms['MAE'] - 10

In [72]:
df_anoms['extra_MAE_cost'] = df_anoms['extra_MAE']*9

In [73]:
df_anoms.groupby(df_anoms['Date'].dt.month)['extra_MAE_cost'].sum()

Date
10     506.006368
11    1215.274010
Name: extra_MAE_cost, dtype: float64

In [74]:
df_all.groupby(df_all['Date'].dt.month)['Total_Consumption'].sum().reset_index()['Total_Consumption']*7

0       0.0
1       0.0
2       0.0
3    3960.6
4    6951.0
5    2346.4
Name: Total_Consumption, dtype: float64

In [75]:
638/8862 # 8-15%

0.0719927781539156

### Threshold: 12

In [76]:
df_anoms = df_all[df_all['MAE'] >= 12]
df_anoms.reset_index(drop=True, inplace=True)

In [77]:
df_anoms.tail(2)

,Date,Total_Consumption,Predicted_Consumption,MAE
11,2023-11-29,53.4,27.115414,26.284586
12,2023-11-30,44.1,27.115414,16.984586


In [78]:
df_anoms['extra_MAE'] = df_anoms['MAE'] - 12

In [79]:
df_anoms['extra_MAE_cost'] = df_anoms['extra_MAE']*9

In [80]:
df_anoms.groupby(df_anoms['Date'].dt.month)['extra_MAE_cost'].sum()

Date
10     433.245094
11    1038.251462
Name: extra_MAE_cost, dtype: float64

In [81]:
df_all.groupby(df_all['Date'].dt.month)['Total_Consumption'].sum().reset_index()['Total_Consumption']*7

0       0.0
1       0.0
2       0.0
3    3960.6
4    6951.0
5    2346.4
Name: Total_Consumption, dtype: float64

In [82]:
180/2300 # 8-15%

0.0782608695652174

In [83]:
df_all.columns

Index(['Date', 'Total_Consumption', 'Predicted_Consumption', 'MAE'], dtype='object')

In [84]:
ved = pd.DataFrame(columns=['Date', 'Total_Consumption', 'Predicted_Consumption', 'MAE'])
for i in range(len(df_all['Date'])):
    ved.loc[i,'Date'] = df_all.loc[i,'Date']
    ved.loc[i,'Total_Consumption'] = df_all.loc[i,'Total_Consumption']
    ved.loc[i,'Predicted_Consumption'] = df_all.loc[i,'Predicted_Consumption']
    ved.loc[i,'MAE'] = df_all.loc[i,'MAE']

ved.to_csv('full_pred.csv', index=False) 
